In [159]:
import psycopg2
from configparser import ConfigParser


#function to set a connection

def connection(link,database):                                  
    try:
        # Load the configuration file
        config = ConfigParser()
        config.read(link)

        ## Retrieve the credentials
        host = config.get(database, 'host')
        dbname = config.get(database, 'dbname')
        user = config.get(database, 'user')
        password = config.get(database, 'password')

        # Connect to the database
        conn = psycopg2.connect(
            host=host,
            dbname=dbname,
            user=user,
            password=password)
        cur = conn.cursor()
        conn.set_session(autocommit=True)
        return conn, cur
    
    except psycopg2.Error as e:
        print("Error: could not make connection")
        return print(e)

#Fetching data from the database

def query_to_df(sql_query):
    cur.execute(sql_query)
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(cur.fetchall(),columns=columns)
    return df  

link = "C:/Users/pranuh/Desktop/Jupyter/8 week SQL challenge - Data with Danny/danny_diner.ini"
database = "danny_diner"

conn,cur = connection(link,database)

In [158]:
# To close the connection after querying

cur.close()
conn.close()

### Introduction:
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

### Problem Statement :
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:

- sales
- menu
- members

In [160]:
# Sales Table

query_to_df("select * from sales limit 5")

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3


In [32]:
# Menu Table

query_to_df("Select * from menu  limit 5")

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [20]:
# Members Table

query_to_df("Select * from members  limit 5")

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


#### 1.What is the total amount each customer spent at the restaurant?

In [38]:
query_to_df("""

    SELECT 
        customer_id,
        sum(price) AS amount_spent 
    FROM 
        sales
    JOIN menu 
        ON sales.product_id = menu.product_id 
    GROUP BY 
        customer_id 
    ORDER BY 
        customer_id

""")

,customer_id,amount_spent
0,A,76
1,B,74
2,C,36


#### 2.How many days has each customer visited the restaurant?

In [26]:
query_to_df("""

    SELECT 
        customer_id,
        COUNT(DISTINCT order_date) AS days_visited 
    FROM 
        sales  
    GROUP BY 
        customer_id

""")

,customer_id,days_visited
0,A,4
1,B,6
2,C,2


#### 3.What was the first item from the menu purchased by each customer?

In [64]:
query_to_df("""

    WITH first_item_cte AS
    (
        SELECT customer_id,order_date,product_name,
            DENSE_RANK() OVER( PARTITION BY customer_id ORDER BY order_date) AS  rank
        FROM  
            sales s
        JOIN menu m ON m.product_id = s.product_id
    )
    SELECT 
        customer_id,product_name        
    FROM 
        first_item_cte
    WHERE rank = 1
        
""")

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry


#### 4.What is the most purchased item on the menu and how many times was it purchased by all customers?

In [48]:
query_to_df("""

    SELECT 
        product_name,COUNT(sales.product_id)
    FROM 
        sales
    JOIN menu ON sales.product_id = menu.product_id
    GROUP BY 
        menu.product_name
    ORDER BY count DESC 
    LIMIT 1

""")

,product_name,count
0,ramen,8


#### 5.Which item was the most popular for each customer?

In [65]:
query_to_df("""

    WITH fav_item_cte AS
    (
        SELECT 
          product_name,
          customer_id,
          COUNT(*) AS count ,
          MAX(COUNT(*)) OVER (PARTITION BY customer_id) AS max_count
        FROM 
            sales
        JOIN menu ON sales.product_id = menu.product_id
        GROUP BY 
            customer_id,menu.product_name
    ) 
        
    SELECT 
        customer_id,
        product_name,
        max_count 
    FROM  
        fav_item_cte
    WHERE 
        count = max_count
    
""")

,customer_id,product_name,max_count
0,A,ramen,3
1,B,ramen,2
2,B,curry,2
3,B,sushi,2
4,C,ramen,3


In [77]:
# 5.Which item was the most popular for each customer?

# Approch 2 using Dense Rank

query_to_df("""

    WITH fav_item_cte AS
    (
        SELECT 
            s.customer_id, 
            m.product_name, 
            COUNT(m.product_id) AS order_count,
            DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY COUNT(s.customer_id) DESC) AS rank
        FROM 
            menu AS m
        JOIN sales AS s ON m.product_id = s.product_id
        GROUP BY 
            s.customer_id, m.product_name
    )

    SELECT 
      customer_id, 
      product_name, 
      order_count
    FROM 
        fav_item_cte 
    WHERE 
        rank = 1;

""")

,customer_id,product_name,order_count
0,A,ramen,3
1,B,curry,2
2,B,ramen,2
3,B,sushi,2
4,C,ramen,3


#### 6.Which item was purchased first by the customer after they became a member?

In [21]:
query_to_df("""

    WITH join_date_cte AS
        (
        SELECT m.customer_id,
            s.order_date,
            menu.product_name,
            MIN(order_date) OVER (PARTITION BY m.customer_id) AS first_order_date
        FROM 
            sales s
        JOIN members m ON s.customer_id = m.customer_id
        JOIN menu ON menu.product_id = s.product_id
        WHERE 
            s.order_date >= m.join_date 
        )
    SELECT 
        customer_id,
        order_date,
        product_name
    FROM 
        join_date_cte
    WHERE 
        order_date = first_order_date

""")

,customer_id,order_date,product_name
0,A,2021-01-07,curry
1,B,2021-01-11,sushi


#### 7.Which item was purchased just before the customer became a member?

In [22]:
query_to_df("""

    WITH join_date_cte AS
        (
        SELECT m.customer_id,
            s.order_date,
            menu.product_name,
            MAX(order_date) OVER (PARTITION BY m.customer_id) AS last_order_date
        FROM 
            sales s
        JOIN members m ON s.customer_id = m.customer_id
        JOIN menu ON menu.product_id = s.product_id
        WHERE 
            s.order_date < m.join_date 
        )
    SELECT 
        customer_id,
        order_date,
        product_name
    FROM 
        join_date_cte
    WHERE 
        order_date = last_order_date

""")

,customer_id,order_date,product_name
0,A,2021-01-01,sushi
1,A,2021-01-01,curry
2,B,2021-01-04,sushi


#### 8.What is the total items and amount spent for each member before they became a member?

In [119]:
query_to_df("""

    SELECT 
        m.customer_id,COUNT(DISTINCT(s.product_id)),
        sum(price) AS amount_spent
    FROM 
        members m
    JOIN sales s ON s.customer_id = m.customer_id
    JOIN menu ON menu.product_id = s.product_id
    WHERE 
        order_date < join_date
    GROUP BY 
        m.customer_id
    
""")

,customer_id,count,amount_spent
0,A,2,25
1,B,2,40


#### 9.If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [85]:
query_to_df("""
   
    WITH points_cte AS 
   (
        SELECT 
           product_name,price,product_id,
        CASE
            WHEN product_name = 'sushi' THEN price * 10 * 2
            ELSE price*10
        END AS points
        FROM
            menu
   )
   SELECT customer_id,SUM(points)
   FROM
       points_cte p
    JOIN sales s ON s.product_id = p.product_id
    GROUP BY 
        customer_id
    ORDER BY 
        customer_id
   
""")

,customer_id,sum
0,B,940
1,A,860
2,C,360


####  10.In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [131]:
query_to_df("""

    SELECT 
        m.customer_id,
        SUM ( 
            CASE 
                WHEN product_name = 'sushi' THEN price * 10 * 2
                WHEN order_date BETWEEN join_date AND ( join_date + INTERVAL '6 DAYS' ) THEN price * 10 * 2
                ELSE price * 10
            END
            ) AS points
    FROM members m
    JOIN sales s ON s.customer_id = m.customer_id 
    JOIN menu m2 ON m2.product_id = s.product_id
    WHERE 
        order_date <='2021-01-31'
    GROUP BY
        m.customer_id
    
""")

,customer_id,points
0,A,1370
1,B,820


#### Join All The Things - Recreate the table with: customer_id, order_date, product_name, price, member (Y/N)

In [90]:
query_to_df("""

    SELECT 
        s.customer_id,
        order_date,
        product_name,
        price,
    CASE 
        WHEN join_date <= order_date THEN 'Y'
        ELSE 'N'
    END AS member    
    FROM
       sales s
    JOIN menu m 
        ON m.product_id = s.product_id
    LEFT JOIN members m2 
        ON m2.customer_id = s.customer_id 
    ORDER BY 1,2,3
    
""")

,customer_id,order_date,product_name,price,member
0,A,2021-01-01,curry,15,N
1,A,2021-01-01,sushi,10,N
2,A,2021-01-07,curry,15,Y
3,A,2021-01-10,ramen,12,Y
4,A,2021-01-11,ramen,12,Y
5,A,2021-01-11,ramen,12,Y
6,B,2021-01-01,curry,15,N
7,B,2021-01-02,curry,15,N
8,B,2021-01-04,sushi,10,N
9,B,2021-01-11,sushi,10,Y


#### Rank All The Things
Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

In [108]:
query_to_df("""

    WITH member_cte AS
    (    
    SELECT 
        s.customer_id,
        order_date,
        product_name,
        price,
    CASE 
        WHEN join_date <= order_date THEN 'Y'
        ELSE 'N'
    END AS member    
    FROM
       sales s
    JOIN menu m 
        ON m.product_id = s.product_id
    LEFT JOIN members m2 
        ON m2.customer_id = s.customer_id 
    )
    SELECT *,
    CASE 
        WHEN member = 'Y' THEN DENSE_RANK() OVER( PARTITION BY customer_id,member ORDER BY order_date ) 
        ELSE NULL
    END AS RANK 
    FROM
        member_cte
""")

,customer_id,order_date,product_name,price,member,rank
0,A,2021-01-01,sushi,10,N,NaN
1,A,2021-01-01,curry,15,N,NaN
2,A,2021-01-07,curry,15,Y,1.0
3,A,2021-01-10,ramen,12,Y,2.0
4,A,2021-01-11,ramen,12,Y,3.0
5,A,2021-01-11,ramen,12,Y,3.0
6,B,2021-01-01,curry,15,N,NaN
7,B,2021-01-02,curry,15,N,NaN
8,B,2021-01-04,sushi,10,N,NaN
9,B,2021-01-11,sushi,10,Y,1.0


In [132]:
# To close the connection after querying

cur.close()
conn.close()